In [ ]:
!pip install langchain_core
!pip install langchain_groq

In [ ]:
pip configparser

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
import psycopg2
import os
import configparser

In [20]:
# Function to establish connection with PostgreSQL database using psycopg2
def connect_to_db(host: str, port: str, user: str, passwd: str, db_name: str):
    conn = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=passwd,
        database=db_name
    )
    return conn

In [29]:
# Function to fetch table and column information from the database schema
def get_db_schema(conn, table_name):
    cursor = conn.cursor()
    cursor.execute(f"""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema='public' AND table_name='{table_name}'
    """)
    schema_info = [row[0] for row in cursor.fetchall()]
    print('\nin the function get_db_schema, the schema fetched :: ', schema_info)
    cursor.close()
    return schema_info

In [35]:
def create_sql_chain(conn, target_table, question):
    schema_info = get_db_schema(conn, target_table)
    print('\nin the function create_sql_chain, the schema that was passed to this function :: ', schema_info)

    template = f"""
        Based on the table schema of table '{target_table}', write a SQL query to answer the question. Use the exact column name from the schema.

        Response Format:
            Question: {question}
            SQL Query:
    """

    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatGroq(model="llama3-8b-8192", temperature=0.2, groq_api_key=os.environ['api']['groq_api_key'])

    return (
        RunnablePassthrough(assignments={"schema": schema_info, "question": question})
        | prompt
        | llm
        | StrOutputParser()
    )

In [36]:
# PGEND_POINT = 'databaseinstance.crooqm2eco97.eu-north-1.rds.amazonaws.com' # End_point
# PGDATABASE_NAME = 'TestDB' # Database Name example: youtube_test_db
# PGUSER_NAME = 'postgres' # UserName
# PGPASSWORD = 'Root.123'

In [37]:
# Function to create SQL query generation chain
config = configparser.ConfigParser()
config.read('config.ini')

def main():
    hostname = config['database']['hostname']
    username = config['database']['username']
    password = config['database']['password']
    database_name = config['database']['database_name']
    port = config['database']['port']
    target_table = config['database']['target_table']

    conn = connect_to_db(hostname, port, username, password, database_name)
    print("Connected to the database successfully!")

    user_query = input("Ask your database a question about " + target_table + ": ")

    sql_chain = create_sql_chain(conn, target_table, user_query)
    sql_query = sql_chain.invoke({})

    print(f"Generated SQL Query: {sql_query}")

    conn.close()

In [38]:
if __name__ == "__main__":
    main()

Connected to the database successfully!
Ask your database a question about turbotable: what are the HPT coolant bleed and LPT coolant bleed values for the 10th cycle of engine 2?

in the function get_db_schema, the schema fetched ::  ['index', 'engine_no', 'cycles_completed', 'altitude', 'mach_no', 'throttle_angle', 'fan_inlet_temp', 'lpc_outlet_temp', 'hpc_outlet_temp', 'lpt_outlet_temp', 'fan_inlet_pressure', 'bypass_duct_pressure', 'hpc_outlet_pressure', 'fan_speed', 'core_speed', 'engine_pressure_ratio', 'hpc_outlet_static_pressure', 'fuel_ps30_ratio', 'corrected_fan_speed', 'corrected_core_speed', 'bypass_ratio', 'burner_fuel_air_ratio', 'bleed_enthalpy', 'demanded_fan_speed', 'demanded_corrected_fan_speed', 'hpt_coolant_bleed', 'lpt_coolant_bleed', 'source']

in the function create_sql_chain, the schema that was passed to this function ::  ['index', 'engine_no', 'cycles_completed', 'altitude', 'mach_no', 'throttle_angle', 'fan_inlet_temp', 'lpc_outlet_temp', 'hpc_outlet_temp', 'l

In [40]:
# what are the hpt_coolant_bleed and lpt_coolant_bleed values for the 10th cycle of engine 2?
# what are the HPT coolant bleed and LPT coolant bleed values for the 10th cycle of engine 2?